# Add Message History

`RunnableWithMessageHistory` 允许我们将消息历史记录添加到某些类型的链中，它包装另一个`Runnable`并管理它的消息历史记录。

https://python.langchain.com/docs/expression_language/how_to/message_history/

In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at {ability}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
runnable = prompt | model

In [2]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

# 定义一个函数来获取历史消息
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [3]:
with_message_history.invoke(
    {"ability": "math", "input": "What does cosine mean?"},
    config={"configurable": {"session_id": "abc123"}},
)

AIMessage(content='Cosine is a trigonometric function that calculates the ratio of the adjacent side to the hypotenuse in a right triangle.', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 33, 'total_tokens': 59}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a9fa9779-8218-494a-9b24-d9d5378dbd7c-0')

In [4]:
# Remembers
with_message_history.invoke(
    {"ability": "math", "input": "What?"},
    config={"configurable": {"session_id": "abc123"}},
)

AIMessage(content='Cosine is a mathematical function used to calculate the ratio of the length of the adjacent side to the length of the hypotenuse in a right triangle.', response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 69, 'total_tokens': 100}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6172fe67-b628-41e4-81e3-6b1102a463c0-0')

In [5]:
# New session_id --> does not remember.
with_message_history.invoke(
    {"ability": "math", "input": "What?"},
    config={"configurable": {"session_id": "def234"}},
)

AIMessage(content='How can I assist you with math?', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 30, 'total_tokens': 38}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ee6cd1db-47c6-4e42-a55e-5d8f11cbef93-0')

## 定义历史消息记录的配置参数

下面使用 `user_id` 和 `conversation_id` 作为主键，定义历史消息记录的配置参数。

In [6]:
from langchain_core.runnables import ConfigurableFieldSpec

store = {}


def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    if (user_id, conversation_id) not in store:
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="Conversation ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)

In [8]:
with_message_history.invoke(
    {"ability": "math", "input": "什么是乘法分配律"},
    config={"configurable": {"user_id": "123", "conversation_id": "1"}},
)

AIMessage(content='乘法分配律是指 a × (b + c) = a × b + a × c，其中 a、b、c 是任意实数。', response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 59, 'total_tokens': 96}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a5c7a6ad-0a32-4183-a9af-bb8311deaef9-0')

In [9]:
with_message_history.invoke(
    {"ability": "math", "input": "What"},
    config={"configurable": {"user_id": "123", "conversation_id": "1"}},
)

AIMessage(content='乘法分配律是一个数学原理，它说明了如何在进行乘法运算时，将一个数与括号内的两个数相加后再相乘。', response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 105, 'total_tokens': 154}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e9a095ab-afdb-4440-a906-dce1b3425772-0')

In [10]:
# 使用不同的 conversation_id 无法定位到历史消息，因此这次对话不受历史消息的影响

with_message_history.invoke(
    {"ability": "math", "input": "What"},
    config={"configurable": {"user_id": "123", "conversation_id": "2"}},
)

AIMessage(content='Do you need help with a math problem?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 29, 'total_tokens': 38}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d1020eb4-4e67-4b72-937e-1aef597e153d-0')

## Examples with runnables of different signatures

In [11]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel({"output_message": ChatOpenAI()})


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    output_messages_key="output_message",
)

with_message_history.invoke(
    [HumanMessage(content="What did Simone de Beauvoir believe about free will")],
    config={"configurable": {"session_id": "baz"}},
)

{'output_message': AIMessage(content='Simone de Beauvoir believed in the existence of free will. She argued that individuals have the ability to make choices and act upon them, rather than being determined by external circumstances or societal expectations. Beauvoir emphasized the importance of personal responsibility and the need for individuals to actively engage in the world and create their own meaning and purpose. In her book "The Ethics of Ambiguity," she wrote, "Man is condemned to be free; because once thrown into the world, he is responsible for everything he does." Beauvoir rejected any deterministic or fatalistic views that deny human agency and advocated for the exercise of free will in order to achieve personal and collective liberation.', response_metadata={'token_usage': {'completion_tokens': 133, 'prompt_tokens': 17, 'total_tokens': 150}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f546c036-9b8b-4d8d-b4c

In [12]:
with_message_history.invoke(
    [HumanMessage(content="How did this compare to Sartre")],
    config={"configurable": {"session_id": "baz"}},
)

{'output_message': AIMessage(content='Simone de Beauvoir\'s views on free will closely align with those of Jean-Paul Sartre, her longtime partner and philosophical collaborator. Both Beauvoir and Sartre were existentialists who emphasized human freedom and the ability to make choices. They rejected any notion of determinism or predestination and believed that individuals are responsible for their actions and the meaning they give to their lives.\n\nSartre famously stated in his book "Being and Nothingness" that "Man is condemned to be free," echoing Beauvoir\'s perspective. He argued that human existence precedes essence, meaning that individuals are first thrown into the world without predetermined purposes or meanings. According to Sartre, individuals have the freedom to define their own essence through their actions and choices.\n\nWhile Beauvoir and Sartre shared similar views on free will, they did have some differences in their philosophical perspectives. For example, Beauvoir em

In [13]:
# Message input, message output

RunnableWithMessageHistory(
    ChatOpenAI(),
    get_session_history,
)

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
| RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000014AFFDDC290>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000014AFFDDC800>, openai_api_key=SecretStr('**********'), openai_api_base='https://api.chatanywhere.tech/v1', openai_proxy=''), config_factories=[<function Runnable.with_listeners.<locals>.<lambda> at 0x0000014AFFE2CAE0>]), config={'run_name': 'RunnableWithMessageHistory'}), get_session_history=<function get_session_history at 0x0000014AE32DD9E0>, history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [14]:
# Dict with single key for all messages input, messages output

from operator import itemgetter

RunnableWithMessageHistory(
    itemgetter("input_messages") | ChatOpenAI(),
    get_session_history,
    input_messages_key="input_messages",
)

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  input_messages: RunnableBinding(bound=RunnableLambda(_enter_history), config={'run_name': 'load_history'})
}), config={'run_name': 'insert_history'})
| RunnableBinding(bound=RunnableLambda(itemgetter('input_messages'))
  | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000014AFFDDC0B0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000014AFFE40500>, openai_api_key=SecretStr('**********'), openai_api_base='https://api.chatanywhere.tech/v1', openai_proxy=''), config_factories=[<function Runnable.with_listeners.<locals>.<lambda> at 0x0000014AFFE2ED40>]), config={'run_name': 'RunnableWithMessageHistory'}), get_session_history=<function get_session_history at 0x0000014AE32DD9E0>, input_messages_key='input_messages', history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', descr